# MODEL ARCHITECTURE

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

file_path = 'path/to/dataset'
data = pd.read_csv(file_path)
print(data.head())

In [ ]:
filtered_data = data[~data['class'].isin([0, 7])]
print(filtered_data['class'].value_counts())
filtered_data.drop(columns=['time','label'], inplace=True)

filtered_data.shape

In [ ]:
df = filtered_data.copy()

X = df.drop('class', axis=1)
y = df['class']

In [ ]:
df.describe()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
y = df['class'].values

X_amplified = X * 1e4
X_normalized = (X_amplified - np.mean(X_amplified, axis=0)) / np.std(X_amplified, axis=0)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
X_train.shape

In [ ]:
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(10, 1)),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(6, activation='softmax')
])


In [ ]:
optimizer = SGD(learning_rate=0.0005, momentum=0.9)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50, batch_size=32,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
model.save("model6839.keras")

# GRADIO DEPLOYMENT

In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import gradio as gr

In [ ]:
action_map = {
    1: "Hand at rest",
    2: "Hand clenched in a fist",
    3: "Wrist flexion",
    4: "Wrist extension",
    5: "Radial deviations",
    6: "Ulnar deviations",
}

In [ ]:
def action(e1, e2, e3, e4, e5, e6, e7, e8):
  model = load_model('model6839.keras')
  input_data = np.array([[e1, e2, e3, e4, e5, e6, e7, e8]])
  prediction = model.predict(input_data)
  predicted_class = np.argmax(prediction, axis=-1)
  return action_map.get(predicted_class[0]+1, "Unknown action")

In [ ]:
inputs = [
    gr.Number(label="e1"),
    gr.Number(label="e2"),
    gr.Number(label="e3"),
    gr.Number(label="e4"),
    gr.Number(label="e5"),
    gr.Number(label="e6"),
    gr.Number(label="e7"),
    gr.Number(label="e8"),
]

output = gr.Textbox(label="Prediction")

examples = [
    [-2.00e-05, 1.00e-05, 2.20e-04, 1.80e-04, -1.50e-04, -5.00e-05, 1.00e-05, 0],
    [1.60e-04, -1.00e-04, -2.40e-04, 2.00e-04, 1.00e-04, -9.00e-05, -5.00e-05, -5.00e-05],
    [-1.00e-05, 1.00e-05, 1.00e-05, 0, -2.00e-05, 0, -3.00e-05, -3.00e-05],
]

In [ ]:
def func(e1, e2, e3, e4, e5, e6, e7, e8):
  return action(e1, e2, e3, e4, e5, e6, e7, e8)

In [ ]:
iface = gr.Interface(
    fn=func,
    inputs=inputs,
    outputs=output,
    title="ML Model Predictor",
    examples=examples,
    flagging_options=["Working", "Not Wotking"],
    description="Enter the 8 feature values to get a prediction."
)

In [ ]:
iface.launch(share=True)